In [1]:
import os
import json
import re
from datetime import datetime
from pathlib import Path
from shutil import copyfile
from urllib.parse import quote_plus

import nltk
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from kaggle_secrets import UserSecretsClient
!pip install --quiet langchain langchain-community unstructured[docx]

from sentence_transformers import SentenceTransformer
from transformers import pipeline
# from langchain_unstructured import UnstructuredLoader as UnstructuredFileLoader
from langchain_community.document_loaders import UnstructuredFileLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.6/212.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-stora

2025-07-21 20:52:25.130840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753131145.329392      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753131145.380795      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# === NLTK Setup ===
required_nltk_packages = [
    "punkt", "averaged_perceptron_tagger", "maxent_ne_chunker", "words", "stopwords"
]

def nltk_package_path(package):
    return {
        "punkt": "tokenizers/punkt",
        "averaged_perceptron_tagger": "taggers/averaged_perceptron_tagger",
        "maxent_ne_chunker": "chunkers/maxent_ne_chunker",
        "words": "corpora/words",
        "stopwords": "corpora/stopwords"
    }.get(package, package)

for package in required_nltk_packages:
    try:
        nltk.data.find(f"{nltk_package_path(package)}")
    except LookupError:
        nltk.download(package)

# === Models ===
embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5")
llm = pipeline("text2text-generation", model="google/flan-t5-large", device=0, max_new_tokens=512)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [3]:
# === Config ===
WATCH_FOLDER_ID = "17sSWn0cfX-jEFmgG4Hl-P9_NtG9kqS6D"
DOWNLOAD_PATH = Path("/tmp/gdrive_docs")
DOWNLOAD_PATH.mkdir(parents=True, exist_ok=True)

# === Auth ===
CRED_INPUT = "/kaggle/input/google-cred/credentials.json"
CRED_WORKING = "/kaggle/working/credentials.json"
copyfile(CRED_INPUT, CRED_WORKING)

gauth = GoogleAuth()
gauth.LoadCredentialsFile(CRED_WORKING)
if gauth.credentials is None:
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    gauth.Refresh()
else:
    gauth.Authorize()
gauth.SaveCredentialsFile(CRED_WORKING)
drive = GoogleDrive(gauth)


In [4]:
# === Utility Functions ===
def sanitize_filename(title):
    return re.sub(r'[\\/*?:"<>|]', "_", title)

def chunk_text(text: str, chunk_size=500, chunk_overlap=50) -> list[str]:
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return [doc.page_content for doc in splitter.create_documents([text])]

def generate_qa_pair(chunk: str) -> dict:
    prompt = f"""
You are an expert grant analyst.

From the text below, extract the following fields and return exactly this JSON format:

{{
  "question": "Your inferred grant question (10–15 words)",
  "answer": "A concise summary from the text",
  "category": "One of: program_description, budget, impact, timeline, team, sustainability"
}}

Text:
\"\"\"
{chunk}
\"\"\"
"""
    try:
        result = llm(prompt.strip())[0]['generated_text']
        print("🧠 LLM Raw Output:", result)
        match = re.search(r"\{.*\}", result, re.DOTALL)
        if match:
            qa = json.loads(match.group())
            if all(key in qa for key in ["question", "answer", "category"]):
                return qa
        raise ValueError("Invalid or incomplete JSON output")
    except Exception as e:
        print(f"⚠️ QA Generation Error: {e}")
        return {"question": None, "answer": None, "category": None}



In [5]:
# === Main ===
def check_for_new_files():
    print(f"🔍 Checking for new files at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    file_list = drive.ListFile({'q': f"'{WATCH_FOLDER_ID}' in parents and trashed=false"}).GetList()

    for file in file_list:
        title = file['title']
        file_id = file['id']
        modified_time = file['modifiedDate']

        # Make filename safe
        doc_id = sanitize_filename(title.replace(" ", "_").lower())

        # Skip if structured JSON already exists
        if Path(f"/kaggle/working/{doc_id}_structured.json").exists():
            print(f"✅ Already processed locally: {title}")
            continue

        print(f"⬇ Downloading: {title}")
        file_path = DOWNLOAD_PATH / f"{sanitize_filename(title)}.docx"
        file.GetContentFile(str(file_path), mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document')

        loader = UnstructuredFileLoader(str(file_path))
        pages = loader.load()
        raw_text = pages[0].page_content
        word_count = len(raw_text.split())

        flat_chunks = []

        for i, page in enumerate(pages):
            chunks = chunk_text(page.page_content)
            for j, chunk in enumerate(chunks):
                qa = generate_qa_pair(chunk)
                try:
                    embedding = embedding_model.encode(chunk).tolist()
                except Exception as e:
                    print(f"⚠️ Embedding error: {e}")
                    embedding = None

                chunk_data = {
                    "chunk_id": f"{doc_id}_chunk_{i}_{j}",
                    "text": chunk,
                    "embedding": embedding,
                    "question": qa["question"],
                    "answer": qa["answer"],
                    "word_count": len(chunk.split()),
                    "metadata": {
                        "doc_id": doc_id,
                        "title": title,
                        "file_id": file_id,
                        "modifiedDate": modified_time,
                        "category": qa["category"]
                    }
                }
                flat_chunks.append(chunk_data)

        doc_entry = {
            "doc_id": doc_id,
            "metadata": {
                "title": title,
                "doc_id": doc_id,
                "file_id": file_id,
                "modifiedDate": modified_time,
                "word_count": word_count
            },
            "chunks": flat_chunks
        }

        # Save flat and structured JSONs in /kaggle/working
        with open(f"/kaggle/working/{doc_id}_structured.json", "w") as f:
            json.dump(doc_entry, f, indent=2)
        with open(f"/kaggle/working/{doc_id}_flat.json", "w") as f:
            json.dump(flat_chunks, f, indent=2)

        try:
            os.remove(file_path)
        except Exception as e:
            print(f"⚠️ Failed to delete file: {e}")

    print("✅ Check complete.\n")

if __name__ == "__main__":
    check_for_new_files()


🔍 Checking for new files at 2025-07-21 20:53:05
⬇ Downloading: DATA - Technology in Action 


/tmp/ipykernel_36/3707570630.py:23: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(str(file_path))


🧠 LLM Raw Output:  "question": What is the purpose of this grant?  "answer": Program_description
⚠️ QA Generation Error: Invalid or incomplete JSON output


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 LLM Raw Output:  """ Social Entrepreneurship Incubators for Youth Coding for Social Impact Training Green Entrepreneurship & Renewable Energy Jobs Training Business Accelerators for Adults in Public Housing- StartUp NYCHA Digital and Gamified Business Learning Experiences and Platforms- Journey Platform Project Name & Teaser Description
⚠️ QA Generation Error: Invalid or incomplete JSON output


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🧠 LLM Raw Output:  """ Amazon Accelerator ($100,000) The Pinkerton Foundation ($200,000) """
⚠️ QA Generation Error: Invalid or incomplete JSON output


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

⬇ Downloading: DATA - Cambio Labs - UJC Communications Onboarding - Landing Page


InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)

In [6]:
import os
import shutil
from pathlib import Path
import zipfile

WORKING_DIR = Path("/kaggle/working")
STRUCTURED_DIR = WORKING_DIR / "structured"
FLAT_DIR = WORKING_DIR / "flat"

# Create folders if they don't exist
STRUCTURED_DIR.mkdir(exist_ok=True)
FLAT_DIR.mkdir(exist_ok=True)

# Move JSON files into their respective folders
for file in WORKING_DIR.glob("*.json"):
    if file.name.endswith("_structured.json"):
        shutil.move(str(file), STRUCTURED_DIR / file.name)
    elif file.name.endswith("_flat.json"):
        shutil.move(str(file), FLAT_DIR / file.name)

# Function to zip a folder
def zip_folder(folder_path: Path, zip_path: Path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in folder_path.rglob('*'):
            zipf.write(file, arcname=file.relative_to(folder_path))

# Zip both folders
zip_folder(STRUCTURED_DIR, WORKING_DIR / "structured.zip")
zip_folder(FLAT_DIR, WORKING_DIR / "flat.zip")

print("✅ Folders zipped to:")
print(" - structured.zip")
print(" - flat.zip")


✅ Folders zipped to:
 - structured.zip
 - flat.zip
